# Sample Supplies Sales Query

In [1]:
import pymongo 
from pymongo import MongoClient

In [2]:
client = pymongo.MongoClient("Enter_Name")

In [3]:
# Specify the database name and collection name
database_name = "sample_supplies"
collection_name = "sales"

# Get the specified database and collection
db = client[database_name]
collection = db[collection_name]

# Aggregation pipeline
pipeline = [
    {
        "$group": {
            "_id": "$storeLocation",
            "document_count": {"$sum": 1}
        }
    },
    {
        "$match": {
            "document_count": {"$gt": 0}
        }
    },
    {
        "$lookup": {
            "from": collection_name,
            "localField": "_id",
            "foreignField": "storeLocation",
            "as": "sales"
        }
    },
    {
        "$unwind": "$sales"
    },
    {
        "$match": {
            "sales.customer.gender": "M"
        }
    },
    {
        "$group": {
            "_id": "$_id",
            "total_count": {"$sum": 1}
        }
    }
]

# Execute the aggregation pipeline
result = list(collection.aggregate(pipeline))

# Print the result
print(result)

# Add spacing
print("\n" + "=" * 30 + "\n")

# Print the result in a nice format
for entry in result:
    print(f"Store Location: {entry['_id']}")
    print(f"Total Count (Gender M): {entry['total_count']}")
    print("-" * 30)

# Code Below is to Verify The Count of Documents
#Code Does not need to be run is only there for my personal learning and verification
# # Verifier Aggregation pipeline
# pipeline = [
#     {
#         "$group": {
#             "_id": "$storeLocation",
#             "total_count": {"$sum": 1},
#             "gender_M_count": {
#                 "$sum": {"$cond": [{"$eq": ["$customer.gender", "M"]}, 1, 0]}
#             },
#             "gender_F_count": {
#                 "$sum": {"$cond": [{"$eq": ["$customer.gender", "F"]}, 1, 0]}
#             },
#             "documents": {"$push": "$$ROOT"}
#         }
#     },
#     {
#         "$match": {
#             "$or": [
#                 {"gender_M_count": {"$gt": 0}},
#                 {"gender_F_count": {"$gt": 0}}
#             ]
#         }
#     },
#     {
#         "$unwind": "$documents"
#     },
#     {
#         "$project": {
#             "storeLocation": "$_id",
#             "total_count": "$total_count",
#             "gender_M_count": "$gender_M_count",
#             "gender_F_count": "$gender_F_count",
#             "customer_gender": "$documents.customer.gender"
#         }
#     },
#     {
#         "$sort": {
#             "customer_gender": 1
#         }
#     },
#     {
#         "$group": {
#             "_id": "$storeLocation",
#             "total_count": {"$first": "$total_count"},
#             "details": {"$push": "$$ROOT"}
#         }
#     }
# ]

# # Verifier Execute the aggregation pipeline
# result = list(collection.aggregate(pipeline))

# # Print the Verifier result
# for document in result:
#     print(f"{'=' * 30}")
#     print(f"City: {document['_id']}")
    
#     # Counter for total
#     total_counter = 1
#     # Counter for male
#     male_counter = 1
#     # Counter for female
#     female_counter = 1
    
#     for detail in sorted(document['details'], key=lambda x: x['customer_gender']):
#         print(f"Total Counter: {total_counter}")
#         total_counter += 1
        
#         if detail['customer_gender'] == 'M':
#             print(f"Male Counter: {male_counter}")
#             male_counter += 1
#         elif detail['customer_gender'] == 'F':
#             print(f"Female Counter: {female_counter}")
#             female_counter += 1
        
#         print(detail)
#         print(f"{'-' * 30}")

[{'_id': 'Denver', 'total_count': 762}, {'_id': 'New York', 'total_count': 250}, {'_id': 'London', 'total_count': 402}, {'_id': 'Austin', 'total_count': 336}, {'_id': 'San Diego', 'total_count': 166}, {'_id': 'Seattle', 'total_count': 557}]


Store Location: Denver
Total Count (Gender M): 762
------------------------------
Store Location: New York
Total Count (Gender M): 250
------------------------------
Store Location: London
Total Count (Gender M): 402
------------------------------
Store Location: Austin
Total Count (Gender M): 336
------------------------------
Store Location: San Diego
Total Count (Gender M): 166
------------------------------
Store Location: Seattle
Total Count (Gender M): 557
------------------------------
